In [1]:
from torch import nn
import torch

from common import set_seed
from sample.process_data_sample import preprocess_bnci2014_001_sample
from shallow_convnet import ShallowConvNet

set_seed()
X, y = preprocess_bnci2014_001_sample(tmin=-0.5, tmax=4)
X = X[0:1, :, :]
X, X.shape



Choosing from all possible events
/opt/anaconda3/envs/ShallowConvNet/lib/python3.11/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 48 events (all good), 1.5 – 6 s (baseline off), ~9.1 MiB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/opt/anaconda3/envs/ShallowConvNet/lib/python3.11/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 48 events (all good), 1.5 – 6 s (baseline off), ~9.1 MiB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/opt/anaconda3/envs/ShallowConvNet/lib/python3.11/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 48 events (all good), 1.5 – 6 s (baseline off), ~9.1 MiB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/opt/anaconda3/envs/ShallowConvNet/lib/python3.11/site-packages/moabb

(tensor([[[ 0.5051,  0.1656, -0.2651,  ..., -0.4802, -0.5219, -0.5698],
          [ 0.6479,  0.2506, -0.2819,  ..., -0.2637, -0.5338, -0.7528],
          [ 0.4502,  0.0364, -0.4540,  ..., -0.6055, -0.7902, -0.9222],
          ...,
          [ 0.3554, -0.0706, -0.4786,  ..., -0.6630, -1.0804, -1.3346],
          [ 0.5127,  0.0696, -0.3793,  ..., -0.5060, -0.8401, -1.0024],
          [ 0.2488, -0.1986, -0.5782,  ..., -0.3893, -0.7625, -0.9942]]]),
 torch.Size([1, 22, 1126]))

In [2]:
model = ShallowConvNet(X.shape)
model.mean_pool = nn.Identity()
model.safe_log = nn.Identity()
model.dropout = nn.Identity()
model.final_conv = nn.Identity()
model.final_squeeze = nn.Identity()
model.eval()

logits = model(X)

logits, logits.shape

(tensor([[[[3.7382e-02],
           [4.1538e-02],
           [6.2381e-02],
           ...,
           [5.9442e-04],
           [1.7480e-02],
           [5.4920e-02]],
 
          [[2.3882e-03],
           [1.3183e-02],
           [1.8901e-02],
           ...,
           [1.7211e-04],
           [3.2088e-02],
           [1.0386e-01]],
 
          [[6.1128e-04],
           [7.7381e-03],
           [2.9627e-02],
           ...,
           [1.2694e-04],
           [5.7105e-03],
           [1.1485e-02]],
 
          ...,
 
          [[5.6275e-03],
           [4.0875e-02],
           [9.0893e-02],
           ...,
           [1.2097e-02],
           [9.1894e-03],
           [4.2411e-03]],
 
          [[6.3578e-02],
           [8.6437e-02],
           [1.2350e-01],
           ...,
           [1.1332e-02],
           [1.8631e-03],
           [2.1949e-04]],
 
          [[5.8387e-03],
           [1.5256e-02],
           [4.1555e-02],
           ...,
           [5.4603e-02],
           [1.3113e-01

before mean pool speedup is worked!

mean pool

[1,2,3,4,5,6,7,8,9,10]   (3,1) (2,1)

[1,2,3,4,5,6] [2,3,4,5,6,7] [3,4,5,6,7,8] [4,5,6,7,8,9] [5,6,7,8,9,10]

[1,2,3] [3,4,5] | [2,3,4] [4,5,6] | [3,4,5] [5,6,7] | [4,5,6] [6,7,8] | [5,6,7] [7,8,9] | [6,7,8] [8,9,10]

[1,2,3] [3,4,5] [5,6,7] [7,8,9]

[2,3,4] [4,5,6] [6,7,8] [8,9,10]


[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] (9,1) (3,1)

[1-12] [2-13] [3-14] [4-15] [5-16] [6-17] [7-18] [8-19] [9-20]

[1-9] [4-12] | [2-10] [5-13] | [3-11] [6-14] | [4-12] [7-15] | [5-13] [8-16] | [6-14] [9-17] | [7-15] [10-18] | [8-16] [11-19] | [9-17] [12-20]

窗口从 index 0 开始，每隔 stride=3 取 kernel=9：
[1-9], [4-12], [7-15], [10-18]   → 共 5 个输出

窗口从 index 1 开始：
[2-10], [5-13], [8-16], [11-19]           → 共 4 个输出

窗口从 index 2 开始：
[3-11], [6-14], [9-17], [12-20]           → 共 4 个输出


In [3]:
kernel = 75
stride = 15

fragments = []
# generate fragments
for offset in range(stride):
    frag = logits[:, :, offset:, :]
    fragments.append(frag)

pooled_frags = []
mean_pool = torch.nn.AvgPool2d(kernel_size=(75, 1), stride=(15, 1))
# mean pool
for frag in fragments:
    pooled_frag = mean_pool(frag)
    pooled_frags.append(pooled_frag)


mean_pool speedup is worked!

In [4]:
crop_num = 627
crop_mean_pool_logits = []
for i in range(crop_num):
    index = i % 15
    skip = i // 15
    crop = pooled_frags[index][:,:,skip + 0: skip + 27,:]
    crop_mean_pool_logits.append(crop)

crop167 = crop_mean_pool_logits[167]
crop167,crop167.shape

(tensor([[[[0.1863],
           [0.1698],
           [0.1779],
           ...,
           [0.1369],
           [0.1340],
           [0.1356]],
 
          [[0.0319],
           [0.0329],
           [0.0227],
           ...,
           [0.0338],
           [0.0372],
           [0.0360]],
 
          [[0.2674],
           [0.2183],
           [0.2236],
           ...,
           [0.1244],
           [0.1209],
           [0.1255]],
 
          ...,
 
          [[0.3251],
           [0.2637],
           [0.2624],
           ...,
           [0.1335],
           [0.1315],
           [0.1321]],
 
          [[0.4992],
           [0.3846],
           [0.4030],
           ...,
           [0.1757],
           [0.1710],
           [0.1793]],
 
          [[0.0855],
           [0.0634],
           [0.0648],
           ...,
           [0.0660],
           [0.0636],
           [0.0637]]]], grad_fn=<SliceBackward0>),
 torch.Size([1, 40, 27, 1]))

In [7]:
frag_log_logits = []
for pool_frag in pooled_frags:
    log_logits = torch.log(torch.clamp(pool_frag, min=1e-6))
    frag_log_logits.append(log_logits)

frag_log_logits,frag_log_logits[0].shape

([tensor([[[[-2.9215],
            [-3.0742],
            [-2.9521],
            ...,
            [-3.3449],
            [-3.4709],
            [-3.4192]],
  
           [[-3.6894],
            [-3.9757],
            [-3.6649],
            ...,
            [-3.2697],
            [-3.0489],
            [-2.6201]],
  
           [[-3.2019],
            [-3.4338],
            [-2.8146],
            ...,
            [-2.9370],
            [-3.0182],
            [-2.8948]],
  
           ...,
  
           [[-2.8204],
            [-3.0679],
            [-2.6933],
            ...,
            [-2.6238],
            [-2.4302],
            [-2.3292]],
  
           [[-2.8474],
            [-3.2130],
            [-2.4184],
            ...,
            [-2.5387],
            [-2.5268],
            [-2.4419]],
  
           [[-3.1142],
            [-2.8988],
            [-2.9243],
            ...,
            [-3.4456],
            [-3.2079],
            [-2.9942]]]], grad_fn=<LogBackward0>),
  t

In [8]:
from torch.nn import init

conv_final = torch.nn.Conv2d(40, 4, (27, 1))
init.xavier_uniform_(conv_final.weight)
init.constant_(conv_final.bias, 0)

result_logits = []
for logits in frag_log_logits:
    result = conv_final(logits)
    result_logits.append(result)

result_logits,result_logits[0].shape

([tensor([[[[-3.4647],
            [-2.9410],
            [-2.5492],
            [-2.2743],
            [-2.2646],
            [-1.8980],
            [-2.3805],
            [-2.5408],
            [-2.4075],
            [-1.8397],
            [-2.1824],
            [-2.1061],
            [-1.7403],
            [-2.3554],
            [-2.3025],
            [-1.8695],
            [-1.8621],
            [-2.3092],
            [-2.4753],
            [-2.7344],
            [-2.9862],
            [-3.2730],
            [-3.1816],
            [-3.3059],
            [-3.4787],
            [-2.6828],
            [-2.1465],
            [-2.0290],
            [-2.0487],
            [-2.2598],
            [-2.4987],
            [-3.1005],
            [-3.0375],
            [-2.8080],
            [-2.7030],
            [-2.7225],
            [-2.7400],
            [-2.8224],
            [-3.2084],
            [-2.8828],
            [-2.7801],
            [-2.5028],
            [-2.5229]],
  
       

In [10]:
crop_num = 627
crop_mean_pool_logits = []
for i in range(crop_num):
    index = i % 15
    skip = i // 15
    crop = result_logits[index][:,:,skip + 0: skip + 1,:]
    crop_mean_pool_logits.append(crop)

# crop167 = crop_mean_pool_logits[167]
# crop167,crop167.shape
crop_mean_pool_logits,crop_mean_pool_logits[0].shape,len(crop_mean_pool_logits)



([tensor([[[[-3.4647]],
  
           [[-2.8334]],
  
           [[ 4.0701]],
  
           [[ 5.3102]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.4578]],
  
           [[-2.8334]],
  
           [[ 4.1346]],
  
           [[ 5.3082]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.4263]],
  
           [[-2.8145]],
  
           [[ 4.1733]],
  
           [[ 5.3259]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.3757]],
  
           [[-2.7866]],
  
           [[ 4.1895]],
  
           [[ 5.3625]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.3256]],
  
           [[-2.7583]],
  
           [[ 4.2050]],
  
           [[ 5.4024]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.2892]],
  
           [[-2.7362]],
  
           [[ 4.2346]],
  
           [[ 5.4290]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.2628]],
  
           [[-2.7249]],
  
           [[ 4.2659]],
  
           [[ 5.4345]]]], grad_fn=<SliceBackward0>),
  tensor([[[[-3.2344]],
  
           [[-2.7196]],
  
        

it works!

In [12]:
result = torch.stack([c.squeeze(0) for c in crop_mean_pool_logits],dim=0)
result,result.shape

(tensor([[[[-3.4647]],
 
          [[-2.8334]],
 
          [[ 4.0701]],
 
          [[ 5.3102]]],
 
 
         [[[-3.4578]],
 
          [[-2.8334]],
 
          [[ 4.1346]],
 
          [[ 5.3082]]],
 
 
         [[[-3.4263]],
 
          [[-2.8145]],
 
          [[ 4.1733]],
 
          [[ 5.3259]]],
 
 
         ...,
 
 
         [[[-2.4209]],
 
          [[-1.9306]],
 
          [[ 4.3003]],
 
          [[ 5.7741]]],
 
 
         [[[-2.4617]],
 
          [[-1.9105]],
 
          [[ 4.2931]],
 
          [[ 5.7608]]],
 
 
         [[[-2.5054]],
 
          [[-1.8804]],
 
          [[ 4.2884]],
 
          [[ 5.7463]]]], grad_fn=<StackBackward0>),
 torch.Size([627, 4, 1, 1]))